In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import scipy.io.wavfile

In [2]:
PATH = '/home/daniel/Descargas/DSLWP-B_PI9CAM_2018-10-19T17_53_35_435.4MHz_40ksps_complex.raw'

In [3]:
starts = range(443,3000,600)
length = 60
fs = 40000
fs2 = 12000
freq_lo = -1000
lo = np.exp(1j*2*np.pi*np.arange(fs*length)*freq_lo/fs)

drift = -25/60

for j, start in enumerate(starts):
    with open(PATH, 'rb') as f:
        f.seek(start * fs * 8)
        n = start * fs
        start_drift = n*(n-1)/2
        start_drift = 0
        drifts = np.cumsum(np.arange(length * fs) + start_drift)
        drift_lo = np.exp(-1j*2*np.pi*drifts*drift/fs**2)
        x = np.fromfile(f, dtype = np.complex64, count = length * fs)
        x = scipy.signal.resample(x * lo * drift_lo, length * fs2)
        f = np.fft.fft(x)
        f[length*fs2//2:] = 0
        x = np.fft.ifft(f).real * 1e1 * 2**15
        scipy.io.wavfile.write(f'/tmp/jt4g_{j:02}.wav', fs2, np.round(x).astype(np.int16))